# Modeling 

 - `nn.Sequential`
 - Sub-class of `nn.Module`

In [25]:
import torch
from torch import nn 
import torch.nn.functional as F

# nn의 모듈

In [26]:
nn.Conv2d(
    in_channels=3,
    out_channels=32,
    kernel_size=3,
    stride=1,
    padding=1,
    bias =False
)

Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [27]:
nn.Linear(
    in_features=784,
    out_features=500,
    bias=False
)

Linear(in_features=784, out_features=500, bias=False)

### `nn.Sequential`


In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [29]:
model = nn.Sequential(
    nn.Linear(784, 15),
    nn.Sigmoid(),
    nn.Linear(15, 10),
    nn.Sigmoid()
)

In [30]:
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=15, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=15, out_features=10, bias=True)
  (3): Sigmoid()
)


In [31]:
# !pip install torchsummary

In [32]:
import torchsummary

In [33]:
torchsummary.summary(model, (784,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 15]          11,775
           Sigmoid-2                   [-1, 15]               0
            Linear-3                   [-1, 10]             160
           Sigmoid-4                   [-1, 10]               0
Total params: 11,935
Trainable params: 11,935
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


### `nn.module` sub class

- `__init__()` 에서 Layers를 초기화 함. 
- `forward` 함수를 구현

In [34]:
# (1, 28, 28)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=3, padding=1)     # stride = 1 기본으로 들어감
        self.conv2 = nn.Conv2d(20, 50, kernel_size=3, padding=1)     # stride = 1 기본으로 들어감
        self.fc1 = nn.Linear(4900, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4900)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=1)
                                        # log_softmax 할때는 dim 을 무조건 설정해야한다.

        return x

In [35]:
model = Net()

In [36]:
torchsummary.summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 28, 28]             200
            Conv2d-2           [-1, 50, 14, 14]           9,050
            Linear-3                  [-1, 500]       2,450,500
            Linear-4                   [-1, 10]           5,010
Total params: 2,464,760
Trainable params: 2,464,760
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.20
Params size (MB): 9.40
Estimated Total Size (MB): 9.60
----------------------------------------------------------------


### 간단한 ResNet 구현

In [41]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
                                                                        # kernel_size=1이기 때문에 padding 이 1이 되면 팽창한다.
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1)
                                                                        # kernel_size=1이기 때문에 padding 이 1이 되면 팽창한다.
        self.conv3 = nn.Conv2d(out_channel, out_channel, kernel_size=1, padding=0)
                                                                        # kernel_size=1이기 때문에 padding 이 1이 되면 팽창한다.
        if in_channel != out_channel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self,x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out += self.shortcut(x)
        return out

In [42]:
class ResNet(nn.Module):
    def __init__(self, color='gray'):
        super(ResNet, self).__init__()
        if color == 'gray':
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)

        elif color == 'rgb':
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
            
        self.resblock1 = ResidualBlock(32, 64)
        self.resblock2 = ResidualBlock(64, 64)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 10)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)

        return x

In [43]:
model = ResNet()

In [44]:
torchsummary.summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 14, 14]           2,112
            Conv2d-3           [-1, 64, 14, 14]          36,928
            Conv2d-4           [-1, 64, 14, 14]           4,160
            Conv2d-5           [-1, 64, 14, 14]           2,112
     ResidualBlock-6           [-1, 64, 14, 14]               0
            Conv2d-7           [-1, 64, 14, 14]           4,160
            Conv2d-8           [-1, 64, 14, 14]          36,928
            Conv2d-9           [-1, 64, 14, 14]           4,160
    ResidualBlock-10           [-1, 64, 14, 14]               0
AdaptiveAvgPool2d-11             [-1, 64, 1, 1]               0
           Linear-12                   [-1, 64]           4,160
           Linear-13                   [-1, 10]             650
Total params: 95,690
Trainable params: 